In [1]:
import sys
sys.path.append(r'd:\\Programming\\Python\\Skripsi\\main\\scripts\\action')

import os
import shutil

import data_prep

### Prepare the destination directory

In [11]:
label = ["normal", "glaukoma"]
dataset_names = ["rimone", 'g1020', 'refuge', 'papila']

path_dataset = r'd://Programming/Python/Skripsi/dataset/'
path_rimone = os.path.join(path_dataset, r'RIM-ONE_DL_images/partitioned_by_hospital')
path_g1020 = os.path.join(path_dataset, r'G1020/Images')
path_refuge = os.path.join(path_dataset, r'REFUGE/Images_Square')
path_papila = os.path.join(path_dataset, r'PAPILA/FundusImages')

path_dataset_destination = r'd://Programming/Python/Skripsi/dataset/combined'

In [3]:
data_prep.create_destination_directory(path=path_dataset_destination,
                                       dataset_names=dataset_names,
                                       label_names=label)

{'Success': [],
 'Already Exists': ['d://Programming/Python/Skripsi/dataset/combined\\rimone\\normal',
  'd://Programming/Python/Skripsi/dataset/combined\\rimone\\glaukoma',
  'd://Programming/Python/Skripsi/dataset/combined\\g1020\\normal',
  'd://Programming/Python/Skripsi/dataset/combined\\g1020\\glaukoma',
  'd://Programming/Python/Skripsi/dataset/combined\\refuge\\normal',
  'd://Programming/Python/Skripsi/dataset/combined\\refuge\\glaukoma',
  'd://Programming/Python/Skripsi/dataset/combined\\papila\\normal',
  'd://Programming/Python/Skripsi/dataset/combined\\papila\\glaukoma']}

## RIM-ONE Dataset
### Getting the file name

In [4]:
file_names_raw_rimone = {
    "training_set": {
        "normal": [],
        "glaucoma": []
    },
    "test_set": {
        "normal": [],
        "glaucoma": []
    }
}

### Validate the file count

In [5]:
for dataset_name_key, dataset_name_value in file_names_raw_rimone.items():
    for label_name_key, label_name_value in dataset_name_value.items():
        file_names_raw_rimone[dataset_name_key][label_name_key] = data_prep.get_file_names(path=os.path.join(path_rimone,
                                                                                                             dataset_name_key,
                                                                                                             label_name_key))

In [6]:
for dataset_name_key, dataset_name_value in file_names_raw_rimone.items():
    for label_name_key, label_name_value in dataset_name_value.items():
        print(f"{dataset_name_key[:5]} {label_name_key}",
              f"Total: {len(label_name_value)}",
              sep="\n")

train normal
Total: 195
train glaucoma
Total: 116
test_ normal
Total: 118
test_ glaucoma
Total: 56


### Transform the raw data into cleaned one

In [7]:
file_names_cleaned = {
    label[0]: [],
    label[1]: []
}
for data_type_key, data_type_value in file_names_raw_rimone.items():
    for file_name_key, file_name_value in data_type_value.items():
        if file_name_key == "normal":
            file_names_cleaned[label[0]].extend(file_name_value)
        elif file_name_key == "glaucoma":
            file_names_cleaned[label[1]].extend(file_name_value)

### Check the validity

In [8]:
for label_name, label_file in file_names_cleaned.items():
    print(f"Total {label_name} : {len(label_file)}")

Total normal : 313
Total glaukoma : 172


### Copying the image

In [9]:
for dataset_name_key, dataset_name_value in file_names_raw_rimone.items():
    for label_name_key, label_name_value in dataset_name_value.items():
        if label_name_key == "normal":
            data_prep.copy_files(source_path=os.path.join(path_rimone,
                                                        dataset_name_key,
                                                        label_name_key),
                                destination_path=os.path.join(path_dataset_destination,
                                                            dataset_names[0],
                                                            label[0]),
                                file_names=label_name_value)
        elif label_name_key == "glaucoma":
            data_prep.copy_files(source_path=os.path.join(path_rimone,
                                                        dataset_name_key,
                                                        label_name_key),
                                destination_path=os.path.join(path_dataset_destination,
                                                            dataset_names[0],
                                                            label[1]),
                                file_names=label_name_value)

### Validate the file

In [12]:
for label_name in label:
    for file_name in data_prep.get_file_names(path=os.path.join(path_dataset_destination,
                                                                 dataset_names[0])):
        if file_name in file_names_cleaned[label_name]:
            pass
        else:
            print(f"{file_name} not found in {label_name}")

## G1020 Dataset

## REFUGE Dataset

## PAPILA Dataset